In [1]:
import pandas as pd
import bz2
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [2]:
DATA_FOLDER = 'data/'
RETURN_FILE_DATA = DATA_FOLDER+"known_speaker_male_female.json.bz2"

In [3]:
GENDER_DICT = DATA_FOLDER + "dict_genders.json"
OCC_DICT = DATA_FOLDER + "dict_occupations.json"
CAT_OCC_DICT = DATA_FOLDER + "dict_categories_occupations.json"

In [4]:
with open(GENDER_DICT) as json_file:
    gender_dict = json.load(json_file)
with open(OCC_DICT) as json_file:
    occ_dict = json.load(json_file)
with open(CAT_OCC_DICT) as json_file:
    cat_occ_dict = json.load(json_file)

In [5]:
df = pd.read_json(RETURN_FILE_DATA)

Find the gender in the gender dictionnary

In [6]:
df['gender'] = [gender_dict[x] for x in df.gender.to_list()]

In [7]:
df.shape[0]

68127

Drop the informations that are no use for us

In [8]:
df.drop(columns = ['quoteID','probas','urls','phase'], inplace=True);

Drop the speakers with no occupation

In [9]:
df.dropna(axis = 0, subset = ['occupation'], inplace = True)
df.shape[0]

65950

# **CODE** si on garde que la première occupation

Keep only the first occupation of each speaker and read it thanks to the occupation dictionnary

In [10]:
df['occupation'] = df['occupation'].apply(lambda x: x[0])

In [11]:
for idx, row in df.iterrows():
        if row['occupation'] not in occ_dict.keys():
            df.drop(idx, inplace=True)

In [12]:
df['occupation'] = [occ_dict[x] for x in df.occupation.to_list()]

Create a new colums which puts the occupations into categories of occupations

In [13]:
df['categorie'] = 0

In [14]:
for idx, row in df.iterrows():
    if row['occupation'] not in cat_occ_dict.keys():
        df.categorie[idx]='Autre'
    else:
        df.categorie[idx]=cat_occ_dict[df.occupation[idx]]

<ipython-input-14-e94b9c8c3ae5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categorie[idx]='Autre'
/Users/potdevinjosephine/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-14-e94b9c8c3ae5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categorie[idx]=cat_occ_dict[df.occupation[idx]]


# **CODE** pour les années

Retrieve the year difference between the date of birth of the speaker and the appearence of his citation in the journals

In [15]:
#from datetime import datetime, date, time

In [16]:
df.dropna(axis = 0, subset = ['date_of_birth'], inplace = True)
df['date_of_birth'] = df['date_of_birth'].apply(lambda x: x[0][1:5])

In [17]:
df['year_of_age'] = 0
for idx, row in df.iterrows():
    df['year_of_age'][idx]=(df.date[idx].year-int(df.date_of_birth[idx]))

<ipython-input-17-0105ae3884af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_of_age'][idx]=(df.date[idx].year-int(df.date_of_birth[idx]))


In [18]:
df.drop(columns='date_of_birth', inplace=True)

As, every information needed is now in the DataFrame df, it is useful to sort it by the categories of occupation.

In [67]:
df.sort_values('categorie', inplace=True)
df.shape[0]

58932

In [64]:
df_final = df.groupby(df.categorie)

In [71]:
df_final

# **CODE** si on regarde toutes les occupations

In [20]:
#for idx, row in df.iterrows():
#    x=0
 #   for i in range(len(row['occupation'])):
  #      if row['occupation'][i-x] not in occ_dict.keys():
   #         row['occupation'].remove(row['occupation'][i-x])
    #        x+=1

In [21]:
#for idx, row in df.iterrows():
 #   for i in range(len(row['occupation'])):
  #      row['occupation'][i]=occ_dict[row['occupation'][i]]

In [22]:
#df

# Problemes avec ce code et améliorations a faire

**1-** Il n'y a que des boucles for ou même des doubles boucles for. Le code marche mais (pour moi) il met du temps à run et il n'est surement pas optimisé

**2-** Je ne sais pas exactement quoi faire à la fin: retourner un groupby; retrouner df_final qui a sort les occupations; faire un sort_values sur quelque chose d'autre de plus pertinant?

**3-** Il faut que je fasse la même chose pour quand on garde toutes les occupations d'un speaker. Idée: mettre dans la colonne 'catégorie' une liste des catégories pour chaque occupation et ne garder à la fin dans la cellule, la catégorie qui revient le plus.